# End-to-End Analysis of Kaggle Airbnb Data

launch_buttons:
  colab_url: "https://colab.research.google.com/github/{user_name}/{repo_name}" 

## Set Up
First, let's import our dependencies into the notebook.  We are using numpy and pandas to represent data in a tabular format, we are using sklearn and statsmodels for modeling, and matplotlib and plotly for visiualizations.

In [3]:
# Data Representation
import numpy as np
import pandas as pd

# Modeling
import sklearn
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version
from sklearn.tree import export_graphviz
import statsmodels

# Visualization
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image
import plotly
import plotly.express as px

# Magic Commands
%matplotlib inline

random_state = 42

## Collect the Data

In [4]:
link = 'https://drive.google.com/file/d/1tT0lNiDHwGQPLa3N0zSdhZcJZaa5aqF3/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+link.split('/')[-2]

In [5]:
listings_df = pd.read_csv(path)
listings_df.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07


## Get to Know the Data with Exploratory Data Analysis (EDA)

In [6]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

In [7]:
listings_df["property_type"].value_counts()

House              1733
Apartment          1708
Townhouse           118
Condominium          91
Loft                 40
Bed & Breakfast      37
Other                22
Cabin                21
Bungalow             13
Camper/RV            13
Boat                  8
Tent                  5
Treehouse             3
Chalet                2
Dorm                  2
Yurt                  1
Name: property_type, dtype: int64

In [8]:
listings_df.select_dtypes(exclude='object')

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
0,241032,20160104002432,956883,3.0,3.0,47.636289,-122.371025,4,1.0,1.0,...,95.0,10.0,10.0,10.0,10.0,9.0,10.0,NaN,2,4.07
1,953595,20160104002432,5177328,6.0,6.0,47.639123,-122.365666,4,1.0,1.0,...,96.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,6,1.48
2,3308979,20160104002432,16708587,2.0,2.0,47.629724,-122.369483,11,4.5,5.0,...,97.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,2,1.15
3,7421966,20160104002432,9851441,1.0,1.0,47.638473,-122.369279,3,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
4,278830,20160104002432,1452570,2.0,2.0,47.632918,-122.372471,6,2.0,3.0,...,92.0,9.0,9.0,10.0,10.0,9.0,9.0,NaN,1,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,8101950,20160104002432,31148752,354.0,354.0,47.664295,-122.359170,6,2.0,3.0,...,80.0,8.0,10.0,4.0,8.0,10.0,8.0,NaN,8,0.30
3814,8902327,20160104002432,46566046,1.0,1.0,47.649552,-122.318309,4,1.0,1.0,...,100.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,1,2.00
3815,10267360,20160104002432,52791370,1.0,1.0,47.508453,-122.240607,2,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3816,9604740,20160104002432,25522052,1.0,1.0,47.632335,-122.275530,2,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


In [65]:
listings_df.select_dtypes(include='object').columns

Index(['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'calendar_updated', 'has_availability',
       'calendar_last_s

In [70]:
listings_df['room_type'].value_counts()

Entire home/apt    2541
Private room       1160
Shared room         117
Name: room_type, dtype: int64

In [9]:
listings_df.select_dtypes(exclude='object').columns

Index(['id', 'scrape_id', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'reviews_per_month'],
      dtype='object')

In [10]:
non_object_columns = listings_df.select_dtypes(exclude='object').columns
non_object_columns

Index(['id', 'scrape_id', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'reviews_per_month'],
      dtype='object')

In [11]:
listings_df['bathrooms'].astype('object')

0       1.0
1       1.0
2       4.5
3       1.0
4       2.0
       ... 
3813    2.0
3814    1.0
3815    1.0
3816    1.0
3817    1.5
Name: bathrooms, Length: 3818, dtype: object

In [12]:
non_object_columns = [column for column in non_object_columns if not 'id' in column or not 'availability' in column]
non_object_columns

['id',
 'scrape_id',
 'host_id',
 'host_listings_count',
 'host_total_listings_count',
 'latitude',
 'longitude',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'square_feet',
 'guests_included',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'license',
 'calculated_host_listings_count',
 'reviews_per_month']

In [13]:
listings_df[non_object_columns].describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,...,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,...,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,...,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,...,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,...,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,...,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,...,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


In [14]:
listings_df.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,...,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,...,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,...,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,...,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,...,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,...,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,...,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


In [15]:
px.histogram(listings_df, x="bedrooms", title="Histogram of Bedrooms", labels={'bedrooms':"Number of Bedrooms"})

In [16]:
beds_baths = listings_df[['bedrooms', 'bathrooms']]
beds_baths['bathrooms'] = beds_baths['bathrooms'].astype('object')

In [17]:
px.histogram(beds_baths.dropna(), x="bedrooms", color="bathrooms", nbins=5)

In [18]:
px.histogram(beds_baths.dropna(), x="bathrooms", color="bedrooms", nbins=5, title="Histogram of Bedrooms with Bathrooms",)

In [19]:
px.histogram(listings_df, x="price", title="Histogram of Listing Prices", labels={'price':"Price Per Night in $"}, nbins=20)

In [20]:
### Split 

In [21]:
listings_df.shape

(3818, 92)

In [22]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(listings_df, test_size=0.2, random_state=random_state)

In [23]:
train_set.shape

(3054, 92)

test_set.shape

## Discover and Visualize the Data

In [146]:
listings = train_set.copy()

In [147]:
px.scatter(listings, x="longitude", y="latitude")

In [148]:
px.scatter_geo(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                projection="natural earth",
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )

In [149]:
listings['latitude'].mean()

47.628755346729

In [150]:
px.scatter_mapbox(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                #projection="natural earth",
                mapbox_style='open-street-map',
                zoom=9,
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )

In [151]:
listings['price'].dtype

dtype('O')

In [152]:
listings['price'].iloc[0] # get first item in the price column

'$35.00'

In [153]:
listings['price'].sort_values().iloc[0:10] # get first ten items in the price column sorted from greatest to least

3122    $1,000.00
1154      $100.00
3599      $100.00
1646      $100.00
1126      $100.00
1774      $100.00
2194      $100.00
2676      $100.00
3743      $100.00
2460      $100.00
Name: price, dtype: object

> We see that the prices are strings and contain the characters: "$", ",", and "."

> If we would like to treat this feature as a continuous variable, we will need to clean it

In [154]:
listings['price_float'] = listings['price'].replace('[\$\,]',"",regex=True).astype(float)
listings['price_float'].dtype

dtype('float64')

In [158]:
px.scatter_mapbox(listings.sort_values(by=['price_float'],ascending=False).head(500), 
                lat="latitude", 
                lon="longitude", 
                mapbox_style='carto-positron',
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()},
                color="price_float",
                size="accommodates",
                zoom=11,
                size_max=10,
                color_continuous_scale=px.colors.cyclical.IceFire,
                hover_data=['bedrooms', 'bathrooms', 'price'],
                title="Listing Locations by Price"
                )

In [156]:
corr_matrix = listings.corr()

In [157]:
corr_matrix["price_float"].sort_values(ascending=False)

price_float                       1.000000
accommodates                      0.650923
bedrooms                          0.622058
beds                              0.591919
bathrooms                         0.514807
square_feet                       0.475637
guests_included                   0.383461
host_total_listings_count         0.101474
host_listings_count               0.101474
review_scores_location            0.070322
review_scores_rating              0.059563
review_scores_cleanliness         0.056968
review_scores_accuracy            0.027898
minimum_nights                    0.017593
review_scores_communication       0.007284
review_scores_checkin             0.007126
maximum_nights                   -0.003205
latitude                         -0.010669
availability_365                 -0.021328
review_scores_value              -0.039537
id                               -0.041048
availability_30                  -0.047732
host_id                          -0.053910
calculated_

In [102]:
listings['accommodates']

700     1
2045    2
644     2
2810    3
2206    2
       ..
1130    2
1294    3
860     2
3507    4
3174    2
Name: accommodates, Length: 3054, dtype: int64

In [103]:
listings['square_feet']

700    NaN
2045   NaN
644    NaN
2810   NaN
2206   NaN
        ..
1130   NaN
1294   NaN
860    NaN
3507   NaN
3174   NaN
Name: square_feet, Length: 3054, dtype: float64

In [159]:
px.scatter(listings, x="price", y="bedrooms", title="Price vs. Bedrooms")

In [160]:
listings['price_float'].head()

700      35.0
2045     66.0
644      80.0
2810    150.0
2206     57.0
Name: price_float, dtype: float64

In [161]:
px.scatter_matrix(listings, 
    dimensions=['accommodates', 'bathrooms','bedrooms', 'beds','price'], 
    )

In [128]:
help(px.parallel_categories)

Help on function parallel_categories in module plotly.express._chart_types:

parallel_categories(data_frame=None, dimensions=None, color=None, labels={}, color_continuous_scale=None, range_color=None, color_continuous_midpoint=None, title=None, template=None, width=None, height=None, dimensions_max_cardinality=50)
        In a parallel categories (or parallel sets) plot, each row of
        `data_frame` is grouped with other rows that share the same values of
        `dimensions` and then plotted as a polyline mark through a set of
        parallel axes, one for each of the `dimensions`.
        
    Parameters
    ----------
    data_frame: DataFrame or array-like or dict
        This argument needs to be passed for column names (and not keyword
        names) to be used. Array-like and dict are tranformed internally to a
        pandas DataFrame. Optional: if missing, a DataFrame gets constructed
        under the hood using the other arguments.
    dimensions: list of str or int, or

In [134]:
px.parallel_categories(listings.sort_values(by=['price_float'], ascending=False).head(25), 
                        dimensions=['accommodates', 'bathrooms','bedrooms', 'beds', 'zipcode'],
                        color='price_float', 
                        color_continuous_scale=px.colors.sequential.Inferno
                        )

## Prepare the Data for Machine Learning

In [137]:
train_set.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [139]:
listings = train_set.drop(["price"], axis=1) # drop target labels for training set (price and price float)
listings_labels = train_set[["price"]].copy()
listings_labels['price_float'] = train_set['price'].replace('[\$\,]',"",regex=True).astype(float)

In [142]:
sample_incomplete_rows = listings[listings.isnull().any(axis=1)]
sample_incomplete_rows.shape

(3054, 91)

### Impute Missing Data

## Select & Fit a Model

### Linear Regression w/`statsmodels`

In [ ]:
import statsmodels.api as sm

In [ ]:
olsmod = sm.OLS(y2,x2)
olsres = olsmod.fit()

In [ ]:
print(olsres.summary())

In [102]:
px.scatter_mapbox(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                #projection="natural earth",
                mapbox_style='basic',
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )

In [100]:
px.scatter_mapbox(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                #projection="natural earth",
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )